In [12]:
# 응용실습
import requests as req
url = "https://api.bithumb.com/v1/candles/minutes/1?market=KRW-BTC&count=1"
d_url = "https://api.bithumb.com/v1/candles/days?market=KRW-BTC&count=1"
ori = req.get(url).json()[0]
ori2 = req.get(d_url).json()[0]

# 시작가
openp = float(ori['opening_price'])
# 최고가 
maxp = float(ori['high_price'])
# 최저가
minp = float(ori['low_price'])
# 변화폭
bandp = maxp - minp


#'''일봉'''
openp2 = float(ori2['opening_price'])
# 최고가 
maxp2 = float(ori2['high_price'])
# 최저가
minp2 = float(ori2['low_price'])
# 변화폭
bandp2 = maxp2 - minp2

print("분기준 시작가:", openp, "\n최고가:", maxp, "\n최저가:", minp, "\n변화폭:", bandp)
print("일기준 시작가:", openp2, "\n최고가:", maxp2, "\n최저가:", minp2, "\n변화폭:", bandp2)
# print("검토:", openp + bandp)
# print("최고가:", maxp)

# 파이썬 외 언어, 삼항연산자 (maxp - (openp + band)) > 0 ?  "떡상중" : "떡락중"
result = "떡상중" if (((openp + bandp)) > maxp) else "떡락중"
result2 = "떡상중" if (((openp2 + bandp2)) > maxp2) else "떡락중"
print("분봉으로 판단:", result)
print("일봉으로 판단:", result2)

분기준 시작가: 96801000.0 
최고가: 96810000.0 
최저가: 96736000.0 
변화폭: 74000.0
일기준 시작가: 101900000.0 
최고가: 102129000.0 
최저가: 88999000.0 
변화폭: 13130000.0
분봉으로 판단: 떡상중
일봉으로 판단: 떡상중


In [ ]:
# 비트코인 원화
import requests
import time

# ──────────────── 설정 ────────────────
USE_UPBIT_AS_PRIMARY = True       # True 추천 (한국 실시간 KRW 가격)
FALLBACK_LAST_RATE = 1450.0       # 매우 오래된 경우 쓸 임의 환율 (2026년 초 평균 근처)

last_usd_krw = None               # 글로벌 캐시

def get_usd_krw():
    """실패 시 None 반환"""
    try:
        # exchangerate.host (현재 제한적임 → 대안으로 바꾸는 게 좋음)
        url = "https://api.exchangerate.host/latest?base=USD&symbols=KRW"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        if 'rates' in data and 'KRW' in data['rates']:
            return float(data['rates']['KRW'])
    except Exception as e:
        print(f"환율 API 오류: {e}")
    return None

def safe_get_usd_krw():
    """안전하게 환율 가져오기 + 캐싱"""
    global last_usd_krw
    
    rate = get_usd_krw()
    if rate is not None and rate > 500:   # 비현실적인 값 방지 (예: 0이나 999999)
        last_usd_krw = rate
        return rate
    
    # 실패 시 마지막 값 사용 (있으면)
    if last_usd_krw is not None:
        print("환율 API 실패 → 마지막 저장값 사용")
        return last_usd_krw
    
    # 처음부터 실패하면 fallback
    print("환율 정보 전혀 없음 → 임시값 사용")
    return FALLBACK_LAST_RATE


# ──────────────── Upbit 버전 (가장 추천) ────────────────
def get_btc_krw_upbit():
    try:
        url = "https://api.upbit.com/v1/ticker?markets=KRW-BTC"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        if resp.status_code == 200 and data:
            return float(data[0]['trade_price'])
    except Exception as e:
        print(f"Upbit 오류: {e}")
    return None


# ──────────────── Binance + 환율 버전 (필요 시) ────────────────
def get_btc_krw_binance():
    try:
        btc_usd = float(requests.get(
            "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT",
            timeout=5
        ).json()['price'])
        
        rate = safe_get_usd_krw()
        return btc_usd * rate
    except Exception as e:
        print(f"Binance 오류: {e}")
        return None


# 테스트용 예시
if __name__ == "__main__":
    for _ in range(100):
        if USE_UPBIT_AS_PRIMARY:
            price = get_btc_krw_upbit()
            source = "Upbit"
        else:
            price = get_btc_krw_binance()
            source = "Binance + cached rate"
        
        if price:
            print(f"[{source}] BTC/KRW : {int(price):,} 원")
        else:
            print("가격 조회 완전 실패")
        
        time.sleep(10)

In [ ]:
# 비트코인 미국달러 환산
import requests
import time

# ──────────────── 설정 ────────────────
def get_btc_usd():
    """Binance에서 BTC/USD 실시간 가격 가져오기"""
    try:
        url = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
        resp = requests.get(url, timeout=5)
        data = resp.json()
        return float(data['price'])
    except Exception as e:
        print(f"Binance API 오류: {e}")
        return None

# 테스트용 예시 (주피터나 .py에서 실행)
if __name__ == "__main__":
    print("BTC/USD 가격 감시 시작 (Binance 기준)")
    print("가격은 약 10초마다 업데이트됩니다.\n")
    
    for i in range(100):
        price = get_btc_usd()
        
        if price is not None:
            # 소수점 2자리까지 표시 (실제 거래소처럼)
            formatted = f"${price:,.2f}"
            print(f"[{i+1:02d}] BTC/USD : {formatted}")
        else:
            print(f"[{i+1:02d}] 가격 조회 실패... 재시도 중")
        
        time.sleep(10)  # 10초 간격 (API 부하 줄이기)

BTC/USD 가격 감시 시작 (Binance 기준)
가격은 약 10초마다 업데이트됩니다.

[01] BTC/USD : $64,875.32
[02] BTC/USD : $64,933.97
[03] BTC/USD : $64,975.70


In [31]:
# 판단하기

# 개인스킬
Me = 'JAVA'

# 모집공고 요구스킬
CompA = ['JAVA', 'PYTHON','HTML','Javascript','C','ORACLE']
CompB = ['HTML', 'PyThon', 'C', 'MySQL']
CompC = ['java','html','CSS','Javascript','C++','ORACLE']

print("합격" if Me in CompA else "불합격")
print("합격" if Me in CompB else "불합격")
print("합격" if Me in CompC else "불합격")

합격
불합격
불합격


In [23]:
# 정제하기(국제우주정거장 인공위성 실시간 위치)
import requests as req
import webbrowser as wb

# 위성위치
url = "http://api.open-notify.org/iss-now.json"
# 탑승자 명단
url2 = "http://api.open-notify.org/astros.json"

result = req.get(url).json()
lat = result['iss_position']['latitude'] # 위도
lon = result['iss_position']['longitude'] # 경도

result2 = req.get(url2).json()["number"]

url = f"https://www.google.com/maps?q={lat}, {lon}"

print(f"현재 {result2}명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다. 위도: {lat}, 경도: {lon}")
wb.open(url)

현재 12명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다. 위도: -5.1098, 경도: 35.2444


True

gio: https://www.google.com/maps?q=-5.1098, 35.2444: Operation not supported


In [ ]:
# 다음은 특정지역에 대한 날씨 API를 찾을 수 있는 링크이다. 
url = "https://api.open-meteo.com/v1/forecast?latitude=37.5665&longitude=126.9780&current_weather=true"
# 다음 주소를 힌트삼아 딕셔너리 데이터형으로 분석하고 의미있는 결과를 출력하라. 

In [27]:
import requests as req

# 서울 위도/경도 기반 Open-Meteo API
url = "https://api.open-meteo.com/v1/forecast?latitude=37.5665&longitude=126.9780&current_weather=true"

# API 요청
data = req.get(url).json()

# current_weather 딕셔너리 추출
weather = data["current_weather"]

# 값 추출
temperature = weather["temperature"]        # 기온 (℃)
windspeed = weather["windspeed"]             # 풍속 (m/s)
winddirection = weather["winddirection"]     # 풍향 (°)
weathercode = weather["weathercode"]         # 날씨 코드
time = weather["time"]                       # 관측 시간

# 날씨 코드 해석용 딕셔너리
weather_desc = {
    0: "맑음",
    1: "대체로 맑음",
    2: "부분적으로 흐림",
    3: "흐림",
    45: "안개",
    48: "착빙 안개",
    51: "이슬비",
    61: "비",
    71: "눈",
    80: "소나기"
}

# 결과 출력
print("관측 시각 :", time)
print("기온 :", temperature, "℃")
print("풍속 :", windspeed, "m/s")
print("풍향 :", winddirection, "°")
print("날씨 :", weather_desc.get(weathercode, "알 수 없음"))

관측 시각 : 2026-02-06T05:45
기온 : -4.8 ℃
풍속 : 9.6 m/s
풍향 : 304 °
날씨 : 맑음


In [30]:
import requests as req

# 1) 기본 API 호출 — 전체 부작용 반응 건수
base_url = "https://api.fda.gov/drug/event.json?count=patient.reaction.reactionmeddrapt.exact"
data = req.get(base_url).json()

results = data["results"]

# 필요한 부작용을 찾는 helper 함수
def find_count(term):
    for item in results:
        if item["term"] == term:
            return item["count"]
    return 0

# 🟡 NAUSEA 보고 건수
nausea_count = find_count("NAUSEA")

# 🟡 VOMITING(구토) 보고 건수
vomit_count = find_count("VOMITING")

# 🟡 COVID-19 관련 보고 건수
covid_count = find_count("COVID-19") + find_count("COVID 19")

# 🟡 MYOCARDIAL INFARCTION(심근경색)
mi_count = find_count("MYOCARDIAL INFARCTION")

# 2) 비교 판단
comparison = "코로나가 더 많음" if covid_count > mi_count else "심근경색이 더 많음"

# 3) NAUSEA vs VOMITING 비교
if nausea_count > vomit_count:
    nv_msg = f"NAUSEA가 더 많음 (차이: {nausea_count - vomit_count} 건)"
elif vomit_count > nausea_count:
    nv_msg = f"VOMITING이 더 많음 (차이: {vomit_count - nausea_count} 건)"
else:
    nv_msg = "NAUSEA와 VOMITING은 동일한 건수"

# 4) meta 정보의 disclaimer에 "FDA" 포함 여부 확인
disclaimer = data.get("meta", {}).get("disclaimer", "")
contains_fda = "FDA" in disclaimer

# 출력
print("📊 NAUSEA 보고 건수   :", nausea_count)
print("📊 COVID-19 보고 건수:", covid_count)
print("📊 심근경색 보고 건수 :", mi_count)
print("🔍 비교 판단         :", comparison)
print("📊 NAUSEA vs VOMITING :", nv_msg)
print("📍 Disclaimer 안에 'FDA' 포함 여부:", contains_fda)


📊 NAUSEA 보고 건수   : 752660
📊 COVID-19 보고 건수: 171703
📊 심근경색 보고 건수 : 171221
🔍 비교 판단         : 코로나가 더 많음
📊 NAUSEA vs VOMITING : NAUSEA가 더 많음 (차이: 305998 건)
📍 Disclaimer 안에 'FDA' 포함 여부: True
